In [1]:
import rasterio
import affine
from rasterio import plot
import matplotlib.pyplot as plt
import numpy as np
np.seterr(divide='ignore', invalid='ignore') #Il se peut qu'il y ait des divisions par 0, numpy retourne 0 dans ces cas.
%matplotlib inline

In [2]:
IMAGE_PIXEL = 90

In [3]:
imagePath = '../Données/S1B_IW_GRDH_1SDV_20200414T223607_20200414T223641_021146_028202_F368.SAFE/measurement/'
vh = rasterio.open(imagePath+'s1b-iw-grd-vh-20200414t223607-20200414t223641-021146-028202-002.tiff', driver='Gtiff') 
vv = rasterio.open(imagePath+'s1b-iw-grd-vv-20200414t223607-20200414t223641-021146-028202-001.tiff', driver='Gtiff')

c:\ihm\miniconda\envs\env_tb\lib\site-packages\rasterio\__init__.py:219: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [4]:
def make_blue(vv, vh):
    return (vv + vh // 2) // vh 

In [5]:
def split_band(band, length, width):
    """Split une bande de Sentinel-1 en bande plus petite"""
    result = []
    for x in range(length):
        columns = []
        for y in range(width):
            lines = []
            for i in range(IMAGE_PIXEL):
                cells = []
                for j in range(IMAGE_PIXEL):
                    cells.append(band[i + IMAGE_PIXEL * x][j + IMAGE_PIXEL * y])
                lines.append(cells)
            columns.append(lines)
        result.append(columns)
    return result

In [6]:
#No merit : https://stackoverflow.com/questions/20677795/how-do-i-compute-the-intersection-point-of-two-lines
def line(p1, p2):
    A = (p1[1] - p2[1])
    B = (p2[0] - p1[0])
    C = (p1[0]*p2[1] - p2[0]*p1[1])
    return A, B, -C

def intersection(L1, L2):
    D  = L1[0] * L2[1] - L1[1] * L2[0]
    Dx = L1[2] * L2[1] - L1[1] * L2[2]
    Dy = L1[0] * L2[2] - L1[2] * L2[0]
    x = Dx / D
    y = Dy / D
    return x,y

def get_gcps(gcps, x, y):
    start_x = IMAGE_PIXEL * x
    start_y = IMAGE_PIXEL * y
    base_coordinates = {}
    print(start_x, start_y)
    for gcp in gcps:
        if gcp.row <= start_y and gcp.col <= start_x:
            base_coordinates['tl'] = gcp
        elif gcp.row <= start_y and gcp.col >= start_x + IMAGE_PIXEL and (not 'tr' in base_coordinates or
                                                                             base_coordinates['tr'].row < gcp.row):
            base_coordinates['tr'] = gcp
        elif gcp.row >= start_y + IMAGE_PIXEL and gcp.col <= start_x and (not 'bl' in base_coordinates or
                                                                             base_coordinates['bl'].col < gcp.col):
            base_coordinates['bl'] = gcp
        elif gcp.row >= start_y + IMAGE_PIXEL and gcp.col >= start_x + IMAGE_PIXEL and not 'br' in base_coordinates:
            base_coordinates['br'] = gcp
        if all(key in base_coordinates for key in ['tl', 'tr', 'bl', 'br']):
            break
    print(base_coordinates)
    result = []
    id = 0
    print(start_x, start_y)
    for coordonnee_x in range(start_x, start_x + IMAGE_PIXEL  + 1):
        for coordonnee_y in range(start_y, start_y + IMAGE_PIXEL + 1):
            bt_col = coordonnee_x - base_coordinates['tl'].col
            ratio_col = base_coordinates['tr'].col - base_coordinates['tl'].col
            bt_x1 = base_coordinates['tl'].x + (base_coordinates['tr'].x - base_coordinates['tl'].x) * bt_col/ratio_col
            bt_y1 = base_coordinates['tl'].y + (base_coordinates['tr'].y - base_coordinates['tl'].y) * bt_col/ratio_col

            bt_x2 = base_coordinates['bl'].x + (base_coordinates['br'].x - base_coordinates['bl'].x) * bt_col/ratio_col
            bt_y2 = base_coordinates['bl'].y + (base_coordinates['br'].y - base_coordinates['bl'].y) * bt_col/ratio_col

            vector_col = (bt_x2 - bt_x1, bt_y2 - bt_y1)
            ratio_col = vector_col[1] * bt_x1 + vector_col[0] * bt_y1

            bt_row = coordonnee_y - base_coordinates['tl'].row
            ratio_row = base_coordinates['bl'].row - base_coordinates['tl'].row
            bt_x3 = base_coordinates['tl'].x + (base_coordinates['bl'].x - base_coordinates['tl'].x) * bt_row/ratio_row
            bt_y3 = base_coordinates['tl'].y + (base_coordinates['bl'].y - base_coordinates['tl'].y) * bt_col/ratio_row

            bt_x4 = base_coordinates['tr'].x + (base_coordinates['br'].x - base_coordinates['tr'].x) * bt_row/ratio_row
            bt_y4 = base_coordinates['tr'].y + (base_coordinates['br'].y - base_coordinates['tr'].y) * bt_row/ratio_row

            L1 = line([bt_x1, bt_y1], [bt_x2, bt_y2])
            L2 = line([bt_x3, bt_y3], [bt_x4, bt_y4])

            x, y = intersection(L1, L2)

            id += 1
            result.append(rasterio.control.GroundControlPoint(row=coordonnee_x - start_y, col=coordonnee_y - start_x, 
                                                              x=x, y=y, id=id))
    return result

In [7]:
import datetime
from rasterio import transform
def create_rgb_image(vv, vh, gcps, gcp_crs, dtype='uint16', 
                          name=datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")):
    """Créée des images plus petites pouvant être mieux traitées par le réseau de neurones"""
    #image_length = len(vv) // IMAGE_PIXEL
    #image_width = len(vv[0]) // IMAGE_PIXEL
    #smaller_vv = split_band(vv, image_length, image_width)
    #print("Fin bande verte")
    #smaller_vh = split_band(vh, image_length, image_width)
    #print("Fin bande rouge")
    #for x in [0, 279]:
    #    for y in [0, 252]:
    #        chang = transform.from_gcps(gcps) * (y * IMAGE_PIXEL,x * IMAGE_PIXEL) #Pas trouvé mieux pour l'instant
    #        transformer = affine.Affine(10.0, 0, chang[0], 0.0, -10.0, chang[1])
    #        smaller_image= rasterio.open(name + '_' + str(x) + '_' + str(y) + '.tiff', 'w', driver='Gtiff', 
    #                      width=IMAGE_PIXEL, height=IMAGE_PIXEL, count=3, crs=gcp_crs, gcps=get_gcps(gcps, x, y),
    #                      #transform=transformer, 
    #                      dtype=dtype
    #                     )
    #        smaller_image.write(make_blue(np.array(smaller_vv[x][y]), np.array(smaller_vh[x][y])), 3)
    #        smaller_image.write(smaller_vh[x][y], 2)
    #        smaller_image.write(smaller_vv[x][y], 1)
    #        smaller_image.close()
    #        print("fin", x, y)
    print(len(vh[0]))
    print(len(vv))
    temp = rasterio.open('output/' + str(name) + '.tiff', 'w', driver='Gtiff', 
                          width=len(vh[0]), height=len(vv), count=3, crs=gcp_crs, gcps=gcps,
                          dtype=dtype
                         )
    temp.write(make_blue(np.array(vv), np.array(vh)), 3)
    temp.write(vh, 2)
    temp.write(vv, 1)
    print("Fichier tiff créé.")

In [8]:
from rasterio import transform
_vv = vv.read(1)
gcps, gcp_crs = vv.gcps
print(vv.height, vv.height % IMAGE_PIXEL)
vv.close()
del vv
_vh = vh.read(1)
dtype = vh.dtypes[0]
print(vh.width, vh.width % IMAGE_PIXEL)
vh.close()
del vh

create_rgb_image(_vv, _vh, gcps, gcp_crs, dtype, name="TempRGB")

22738 58
25118 8
25118
22738


c:\ihm\miniconda\envs\env_tb\lib\site-packages\rasterio\__init__.py:229: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  **kwargs)


Fichier tiff créé.


In [9]:
import os, gdal

out_path = 'output/sentinel1/'
output_filename = 'tile_'

in_file = 'output/TempRGB.tiff'

tile_size_x = IMAGE_PIXEL
tile_size_y = IMAGE_PIXEL

ds = gdal.Open(in_file)
band = ds.GetRasterBand(1)
xsize = band.XSize
ysize = band.YSize

for i in range(0, xsize, tile_size_x):
    for j in range(0, ysize, tile_size_y):
        com_string = "gdal_translate -of GTIFF -srcwin " + str(i) + ", " + str(j) + ", " + str(tile_size_x) + \
            ", " + str(tile_size_y) + " " + str(in_file) + " " + str(out_path) + str(output_filename) + \
            str(int(i/IMAGE_PIXEL)) + "_" + str(int(j/IMAGE_PIXEL)) + ".tiff"
        os.system(com_string)